# NL Grids
NL Grids genereert ruimtelijk complementaire hexagon grids gebaseerd op en geldig voor het Rijksdriehoekstelsel.

## Documentatie
Zie [NL Grids op GitHub](https://github.com/bleutzinn/NL-Grids)

## Copyright en contact
Ron Wardenier, ron@rwgc.nl

In [ ]:
########################################################################################################################

# Oppervlakte eenheid ('m2', 'ha' of 'km2')
eenheid = 'km2'

# Oppervlakte per cel of hexagon (geheel of gebroken getal)
oppervlakte = 100

# Grid gebied (naam van gebied conform WFS Bestuurlijke grenzen)
gebied = 'Nederland'

# Trim grid ('ja' of 'nee')
trim = 'ja'

# Export format ('gml' of 'shp')
export = 'gml'


########################################################################################################################

opp_factor = 0
if eenheid == 'm2':
    opp_factor = 1
elif eenheid == 'ha':
    opp_factor = 10000
elif eenheid == 'km2':
    opp_factor = 1000000
else:
    print('Ongeldige oppervlakte eenheid opgegeven')

    
if opp_factor > 0:
    
    # Initialisatie

    %matplotlib inline
    import time
    import math
    import geopandas as gpd
    import matplotlib as mpl
    import matplotlib.pyplot as plt
    from shapely.geometry import Point, Polygon, MultiPolygon, shape
    from shapely.affinity import translate
    from shapely.ops import unary_union
    from requests import Request
    from owslib.wfs import WebFeatureService
    from owslib.etree import etree
    from owslib.fes import PropertyIsEqualTo
    import fiona
    
    start = time.perf_counter()

    # RD-stelsel oorsprong
    rdx0 = 155000
    rdy0 = 463000
    oorsprong = gpd.GeoDataFrame([[Point([rdx0, rdy0])]],columns=['geometry'])

    # Cel oppervlakte in vierkante meters
    cel_opp = oppervlakte * opp_factor

    # Lengte van een hexagon zijde
    l = math.sqrt((2 * cel_opp) / (3 * math.sqrt(3)))

    # Bepaal de hexagon middelpunt afstanden
    hs = 1.5 * l
    vs = math.sqrt(3) * l

    # Maak hexagon met als middelpunt (155000 463000)
    p = Polygon([[rdx0 + math.cos(math.radians(angle)) * l,
                  rdy0 + math.sin(math.radians(angle)) * l]
                 for angle in range(0, 360, 60)])
    
    print("Download van gebied(en) gestart")

    wfs_url = 'https://geodata.nationaalgeoregister.nl/bestuurlijkegrenzen/wfs'

    if gebied == 'Nederland':
        
        namen = [gebied]

        wfs = WebFeatureService(url=wfs_url)
        layer = 'bestuurlijkegrenzen:landsgrens'
        params = dict(service='WFS', version="1.0.0",
                      request='GetFeature',
                      typeName=layer,
                      outputFormat='json')
        q = Request('GET', wfs_url, params=params).prepare().url
        boundary = gpd.read_file(q)['geometry']

    else:

        namen = []
        lijst = {'gemeenten': [], 'provincies': []}
        for gebied in gebied.split(','):
            niveau, naam = gebied.strip().split(' ',1)

            naam = naam.strip()
            namen.append(naam)
            if niveau.lower().strip() == 'gemeente':
                lijst['gemeenten'].append(naam)
            if niveau.lower().strip() == 'provincie':
                lijst['provincies'].append(naam)

        collections = []
        for indeling in lijst:
            
            instelling = indeling[:-1]
            layer = 'bestuurlijkegrenzen:' + indeling
            
            wfs = WebFeatureService(wfs_url, version='1.1.0')

            for naam in lijst[indeling]:
                naam_filter = PropertyIsEqualTo(propertyname=instelling+'naam', literal=naam)

                gebied_gml = wfs.getfeature(
                    typename=layer,
                    filter=etree.tostring(naam_filter.toXML()).decode("utf8"), outputFormat='JSON')

                collections.append(fiona.open(gebied_gml.read()))

        polygons = []

        for collection in collections:
            # append the geometries to the list
            for polygon in collection:
                polygons.append(shape(polygon['geometry']))

        boundary = gpd.GeoSeries(unary_union(polygons))

    print("Download klaar")
    
    filename = "nlgrid_{}_{}{}".format('_'.join(namen), str(oppervlakte), eenheid)
    layer_name = "nlgrid_{}_{}{}".format('_'.join(namen), str(oppervlakte), eenheid)
        
    # Bepaal de bounding box of extent
    minx, miny, maxx, maxy = boundary.total_bounds

    # Construeer rechthoekig grid die de bounding box van het geselecteerde gebied volledig dekt

    nminx = math.floor((minx - rdx0) / hs)
    nmaxx = math.floor((maxx - rdx0) / hs) + 1
    nminy = math.floor((miny - rdy0) / vs)
    nmaxy = math.floor((maxy - rdy0) / vs) + 1

    index_column_name = 'q_r'
    dekking_column_name = 'dekopp' + '_' + eenheid

    polygons = []
    qr_attributes = []

    vshift = nminx % 2 == 0

    print("Genereren van cellen gestart")
    
    for x in range(nminx, nmaxx + 1):
        vshift = not vshift
        for y in range(nminy, nmaxy + 1):
            polygons.append(translate(p, x * hs, y * vs + vshift * 0.5 * vs))
            qr_attributes.append(str(x) + ' ' + str(y * 2 + vshift))
            
            
    print("Genereren klaar")

    grid = gpd.GeoDataFrame({'geometry':polygons})
    grid.crs = "EPSG:28992"
    grid[index_column_name] = qr_attributes
    grid[dekking_column_name] = 0.0

    print("Dekking oppervlakte per cel berekenen gestart")
    
    for i, g1 in enumerate(grid['geometry']):
        for j, g2 in enumerate(boundary):
            grid.loc[i, dekking_column_name] = round(g1.intersection(g2).area / opp_factor, int(math.log10(opp_factor)))

    print("Dekking oppervlakte berekenen klaar")

    # Als trim == 'ja' dan verwijder lege cellen
    if trim.lower() == 'ja':
        
        print("Verwijderen van ongewenste cellen gestart")
        
        grid.drop(grid[grid[dekking_column_name] == 0].index, inplace = True)
        
        
    print("Verwijderen klaar")

    print("Plotten gestart")
    
    figsize = (16,16)
    with plt.style.context(('seaborn', 'ggplot')):
        #ax = fig.add_subplot()
        base = boundary.plot(figsize=figsize, color='white', edgecolor='grey')
        dekking = grid.plot(ax=base, figsize=figsize, color='none', edgecolor='#5f4b8b')
        rd00 = oorsprong.plot(ax=base, figsize=figsize, marker='o', markersize=20, color='#5f4b8b')
        label = "{},{}".format(rdx0, rdy0)

        plt.annotate(label,
                     (rdx0,rdy0),
                     textcoords="offset points",
                     xytext=(0, 10),
                     ha='center')
        
        plt.annotate('NL Grids', xy=(0, 1), xycoords='axes fraction',
             xytext=(30, -30), textcoords='offset points',
             ha="left", va="top", fontsize=42, color='#5f4b8b')
    
        plt.annotate("Cel oppervlakte: {} {}".format(str(oppervlakte), 'ha' if eenheid == 'ha' else 'km$^2$'), xy=(0, 1), xycoords='axes fraction',
             xytext=(30, -76), textcoords='offset points',
             ha="left", va="top", fontsize=15, color='grey')
    
    plt.savefig(filename + '.png')
    plt.show()

    print("Plotten klaar")
    
    print("Export als dataset gestart")
    
    # Bewaar grid in opgegeven format
    if export.lower() == 'gml':
        grid.to_file(filename + '.gml', layer=layer_name, driver='GML')
        print("Grid dataset GML file: {}".format(filename + '.gml'))
    if export.lower() == 'shp':
        grid.to_file(filename + '.shp', layer=layer_name)
        print("Grid dataset shapefile: {}".format(filename))

    
    print("Export klaar")
    
    print("Aantal cellen in grid: {}".format(len(grid)))

    print(f"Run time: {time.perf_counter() - start:0.1f} seconds")
    print("Klaar")